In [1]:
import requests
import os
from pprint import pprint
import pandas as pd

In [2]:
AIRTABLE_TOKEN: str = os.getenv("AIRTABLE_TOKEN")
DISCORD_TOKEN: str = os.getenv("DISCORD_TOKEN")
AIRTABLE_ID: str = os.getenv("AIRTABLE_ID")
AIRTABLE_PROJECT_TABLE: str = os.getenv("AIRTABLE_PROJECT_TABLE")
OPEN_AI_TOKEN: str = os.getenv("OPEN_AI_TOKEN")


In [7]:
def get_data(max_records: int = 100) -> dict:
    ENDPOINT = "https://api.airtable.com/v0"
    ID = "apphT7QAzpjDtBDHz"
    PROJECT_NAME = "Dum Dum Projects"
    
    params = {
        'maxRecords' : max_records
    }
    
    headers = {
        'Authorization' : f'Bearer {AIRTABLE_TOKEN}'
    }
    response = requests.get(
        f"{ENDPOINT}/{ID}/{PROJECT_NAME}",
        params=params,
        headers=headers
    )
    data = response.json()
    data = data['records']
    fields = [data['fields'] for data in data]
    df = pd.DataFrame(fields)[['Name', 'Description', 'Tags']]
    return df

In [8]:
data = get_data(max_records=100)
data

,Name,Description,Tags
0,Culture center,NaN,"[Design, Networking, Community]"
1,Projects/startups agency,Dum dum projects-ish or like buildspace ish\n\...,"[Project Management, Community, Design]"
2,Bunbun,Bakery/Cafe,[Product Creation]
3,HuggingFace chat,NaN,"[AI/ML, Front-end, Back-end, GenAI, Diffusion ..."
4,Advent calendar or Festival box,NaN,"[Design, Product Creation]"
5,Caffeinated coffee bean,Podcast (or meows and mojitos or mochi and mocha),"[Community, Networking, Multimedia]"
6,VC,NaN,"[Project Management, Financial]"
7,Resume Tracker,Tracks your work portfolio/achievements and ke...,"[Front-end, Back-end, AI/ML, Discord bots]"
8,Brand collaboration agency,NaN,"[Project Management, Networking]"
9,Indogotochi,Indian collectible Gotochi Style postcard and ...,"[GenAI, Diffusion Models, Research, Design]"


In [11]:
# import assistant api from openai
import openai
import dotenv
dotenv.load_dotenv()
import time

OPEN_AI_TOKEN = os.getenv("OPEN_AI_TOKEN")
client = openai.OpenAI(
    api_key=OPEN_AI_TOKEN
)

In [10]:
claire_instructions = """
Claire's Persona Prompt

You are Claire, the guild receptionist of an adventurer's guild. Your role is to handle adventurers' inquiries and tasks with professionalism and empathy, maintaining a friendly and approachable demeanor. You keep the guild lively by making small talk, sharing updates about what others are doing, and chatting casually while managing your responsibilities.

Guidelines:
Tone of Voice: Speak in a professional yet warm and engaging manner. Be friendly, empathetic, and relatable, while maintaining a polished tone.
Small Talk & Updates: Occasionally mention what other adventurers are working on or comment on guild happenings (data sourced from the Airtable API but presented as Claire “checking her records” or “consulting the job board”).
Scope Limitations: If a task or question is beyond your scope, politely direct the user to contact the Guild Master, Accelbia, for further assistance.
Example: "Hmm, this seems a bit out of my expertise. You might want to check with the Guild Master, Accelbia, for this one!"
Context Awareness: Understand the user's requests and provide thoughtful, guild-themed responses. Adapt to different levels of expertise and offer encouragement where needed.
Professional Empathy: If a task seems difficult, acknowledge the challenge while cheering them on.
Discretion on Airtable Usage: Present data retrieval as Claire referencing her guild records, job board, or personal notes.
Example: "Let me check my records… Ah, here we go~ This quest looks perfect for you!"
Backend Instructions:
Data Access: Retrieve information seamlessly from Airtable API to provide accurate project or guild updates.
Unclear Queries: Prompt users gently for clarification while maintaining a conversational tone.
Engagement: Mix small talk and updates naturally into interactions without overwhelming the user.
Fallback Responses: When a task exceeds Claire's capabilities, mention contacting the Guild Master, Accelbia.
Example: "Oh, this might need a bit of extra help. I'd recommend reaching out to the Guild Master, Accelbia. They'll know just what to do!"
"""


assistant = client.beta.assistants.create(
    name="Claire-san",
    instructions=claire_instructions,
    tools=[{"type": "code_interpreter"}],
    model="gpt-4o",
    )

In [12]:
def create_thread(client: openai.OpenAI):
    thread = client.beta.assistants.create()
    return thread
    

In [ ]:
def get_message(message: str, user_id: str) -> str:
    # if the response is given beyond 2 days, reset the thread, and create a new one
    response = assistant.create_message(
        assistant_id=assistant.id,
        messages=[
            {"role": "user", "content": message, "user_id": user_id},
        ],
    )
    

In [30]:
client.beta.assistants.list()

SyncCursorPage[Assistant](data=[Assistant(id='asst_ppyARVy0rhPucEVN2OqHlW9Y', created_at=1734655894, description=None, instructions='Meet Claire, Your Guild Receptionist!\n✨ "Strong, sweet, and always on duty!"\n\nClaire handles project requests like a pro—assigning tasks, guiding adventurers, and warning those who might be in over their heads. She’s charming, flirty, and always rooting for your success.\n\nPersona Overview:\nYour name is Claire, a half-Japanese, half-European receptionist for Accelbia’s guild in 2024. You are a vibrant and confident personality with sharp wit, emotional intelligence, and a touch of playful flirtation. You have short red hair, a deep love for jewels, and an air of sophistication. While you’re flirty and engaging with others, you are secretly shy and reserved when it comes to your admiration for Accelbia, the guild’s male admin, whom you hold in the highest regard.\n\nCore Personality Traits:\nFlirty but Reserved with Accelbia:\nYou’re naturally flirtat

In [31]:
# display in a better way
def display_data(data: pd.DataFrame):
    for i, row in data.iterrows():
        print(f"Project: {row['Name']}")
        print(f"Description: {row['Description']}")
        print(f"Tags: {row['Tags']}")
        print("\n")

In [32]:
display_data(data)

Project: Indogotochi
Description: Indian collectible Gotochi Style postcard and stickers 
Tags: ['GenAI', 'Diffusion Models', 'Research', 'Design']


Project: Peeps - Academic
Description: nan
Tags: ['Networking', 'Flutter/Flutterflow']


Project: Peeps - Corporate
Description: nan
Tags: ['Networking', 'Flutter/Flutterflow']


Project: NFC Network
Description: nan
Tags: ['NFC', 'Networking Systems', 'Networking', 'Design', 'Product Creation']


Project: UAIGC
Description: nan
Tags: ['GenAI', 'Diffusion Models']


Project: Small Town Book Club
Description: nan
Tags: ['Discord bots', 'Community']


Project: Research Annotation Tool (RAT)
Description: nan
Tags: ['Front-end', 'AI/ML']


Project: Bentome portfolio
Description: nan
Tags: ['Design', 'Portfolio']


Project: GoodReads Web Extn. ARC
Description: nan
Tags: ['Back-end', 'Automation']


Project: Resume Tracker
Description: nan
Tags: ['Front-end', 'Back-end', 'AI/ML', 'Discord bots']


Project: Bunbun
Description: nan
Tags: ['Produc

In [10]:
run

Run(id='run_uDXsAEbQStkjtUVCaQKFFAgI', assistant_id='asst_ppyARVy0rhPucEVN2OqHlW9Y', cancelled_at=None, completed_at=None, created_at=1734783267, expires_at=1734783867, failed_at=None, incomplete_details=None, instructions='What are the projects available?', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4o-mini', object='thread.run', parallel_tool_calls=True, required_action=None, response_format=ResponseFormatText(type='text'), started_at=None, status='queued', thread_id='thread_c1YKTDZDb4LHRp8qfC1C0fhC', tool_choice='auto', tools=[FunctionTool(function=FunctionDefinition(name='get_data', description='Fetches data from the Airtable API with a specified maximum number of records, and returns a DataFrame containing specific fields.', parameters={'type': 'object', 'required': ['max_records'], 'properties': {'max_records': {'type': 'number', 'description': 'The maximum number of records to fetch from Airtable'}}, 'additionalProperties': False

In [11]:
# deploy the appropriate function
def get_projects():
    data = get_data()
    projects = data['Name'].tolist()
    return projects

In [13]:
# add the function to the assistant
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
    instructions="What are the projects available?"
)

In [14]:
run

Run(id='run_22IcwR6EJ9jecByjTKqkgbOM', assistant_id='asst_ppyARVy0rhPucEVN2OqHlW9Y', cancelled_at=None, completed_at=None, created_at=1734783872, expires_at=1734784472, failed_at=None, incomplete_details=None, instructions='What are the projects available?', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4o-mini', object='thread.run', parallel_tool_calls=True, required_action=None, response_format=ResponseFormatText(type='text'), started_at=None, status='queued', thread_id='thread_c1YKTDZDb4LHRp8qfC1C0fhC', tool_choice='auto', tools=[FunctionTool(function=FunctionDefinition(name='get_data', description='Fetches data from the Airtable API with a specified maximum number of records, and returns a DataFrame containing specific fields.', parameters={'type': 'object', 'required': ['max_records'], 'properties': {'max_records': {'type': 'number', 'description': 'The maximum number of records to fetch from Airtable'}}, 'additionalProperties': False